<a href="https://colab.research.google.com/github/jdasam/ant5015/blob/2024F/notebooks/3rd_week_note_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from pathlib import Path

import torch
import torchaudio

In [2]:
'''
Let's only use testset
'''
!wget http://download.magenta.tensorflow.org/datasets/nsynth/nsynth-test.jsonwav.tar.gz
# !는 주피터에서 터미널 명령어를 사용할때 붙임
# python이 아닌 OS (Linux)에 명령을 내리고 싶을 때 실행


--2024-03-25 04:33:43--  http://download.magenta.tensorflow.org/datasets/nsynth/nsynth-test.jsonwav.tar.gz
Resolving download.magenta.tensorflow.org (download.magenta.tensorflow.org)... 173.194.193.207, 173.194.194.207, 173.194.195.207, ...
Connecting to download.magenta.tensorflow.org (download.magenta.tensorflow.org)|173.194.193.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349501546 (333M) [application/gzip]
Saving to: ‘nsynth-test.jsonwav.tar.gz’

nsynth-test.jsonwav 100%[===================>] 333.31M  14.5MB/s    in 5.5s    

2024-03-25 04:33:49 (60.9 MB/s) - ‘nsynth-test.jsonwav.tar.gz’ saved [349501546/349501546]



In [3]:
# 압축 풀기
!tar -xf nsynth-test.jsonwav.tar.gz

## Make sine wave
- 1 period of angle = 2 * pi
- 220 period of angle = 2 * pi * 220 / sec